As usual, we import Pandas and Numpy.  I'm also going to use a nice function called MonthEnd, which I will explain below.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from pandas.tseries.offsets import MonthEnd

Here I'm reading in CRSP and Compustat data.  Details about these files can be found in the database_construction.pdf file.

In [2]:
stocks = pd.read_feather('crsp_monthly_stocks.feather')
cstat  = pd.read_feather('compustat_annual.feather')

Let's take a look at the stocks dataframe first.  This is data from CRSP, which contains stock returns (RET), closing prices (PRC), volume (VOL), shares outstanding (SHROUT), a code describing the issue type (SHRCD), a code for the primary exchange (EXCHCD), and an industry code (SICCD).

Firms are identified by PERMNO, which remains constant over a firm's life.  Data are monthly, and the date is equal to the last trading day of the month.

In [3]:
stocks.head(10)

,PERMNO,DATE,SHRCD,EXCHCD,SICCD,PRC,VOL,RET,SHROUT
0,10000.0,1985-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000.0,1986-01-31,10.0,3.0,3990.0,-4.375000,1771.0,NaN,3680.0
2,10000.0,1986-02-28,10.0,3.0,3990.0,-3.250000,828.0,-0.257143,3680.0
3,10000.0,1986-03-31,10.0,3.0,3990.0,-4.437500,1078.0,0.365385,3680.0
4,10000.0,1986-04-30,10.0,3.0,3990.0,-4.000000,957.0,-0.098592,3793.0
5,10000.0,1986-05-30,10.0,3.0,3990.0,-3.109375,1074.0,-0.222656,3793.0
6,10000.0,1986-06-30,10.0,3.0,3990.0,-3.093750,1069.0,-0.005025,3793.0
7,10000.0,1986-07-31,10.0,3.0,3990.0,-2.843750,1163.0,-0.080808,3793.0
8,10000.0,1986-08-29,10.0,3.0,3990.0,-1.093750,3049.0,-0.615385,3793.0
9,10000.0,1986-09-30,10.0,3.0,3990.0,-1.031250,3551.0,-0.057143,3793.0


We're going to clean up the data a bit.  The code below does the following:

1. Shift the date so that it is always the last day of the month, rather than the last trading day.  This will make it easier to merge in with other datasets.
2. Take the absolute value of the closing price.  For shares that don't trade, CRSP sets the price equal to the closing bid-ask midpoint, but it makes the price negative as a warning about this.
3. Define market value (MV) as the product of shares outstanding and closing price.
4. Drop shares outstanding, which we won't use again, and the share code.  We use the share code when we download data from WRDS.  Selecting share codes of 10 or 11 means that we will be downloading common equity and not other securities (ETFs, REITS, etc.).  Also drop EXCHCD, SICCD, PRC, and VOL to make the dataframe easier to display.
5. Set the index to PERMNO/DATE.
6. Sort by the index.
7. Look at the dataframe.

In [4]:
stocks['DATE'] = stocks['DATE'] + MonthEnd(0)
stocks['PRC']  = np.abs(stocks['PRC'])
stocks['MV'] = stocks['SHROUT']*stocks['PRC']
stocks.drop(['SHROUT','SHRCD','EXCHCD','SICCD','PRC','VOL'], axis=1, inplace=True)
stocks.set_index(['PERMNO','DATE'], inplace=True)
stocks.sort_index(inplace=True)
stocks.head()

RET       MV
PERMNO  DATE                         
10000.0 1985-12-31       NaN      NaN
        1986-01-31       NaN  16100.0
        1986-02-28 -0.257143  11960.0
        1986-03-31  0.365385  16330.0
        1986-04-30 -0.098592  15172.0

Looks good.  Now let's look at the Compustat data.  This is annual.  It contains a variety of variables that are described in compustat_variables.xlsx.  The one we will use here is earnings before extraordinary items (IB).  For one small purpose I will also look at book equity (SEQ).

Since I used the CRSP/Compustat merged version of Compustat data, I have a variable LPERMNO that is equivalent to the PERMNO variable in CRSP.  Dates in this file represent the last date of the fiscal year.  They are _not_ the dates at which the data became public.

In [5]:
cstat.head(5)

,DATADATE,FYEAR,LPERMNO,AT,CEQ,LT,PSTK,SEQ,IB,CAPX
0,1970-12-31,1970.0,25881.0,33.450,10.544,22.906,0.000,10.544,1.878,2.767
1,1971-12-31,1971.0,25881.0,29.330,8.381,20.948,0.000,8.382,0.138,1.771
2,1972-12-31,1972.0,25881.0,19.907,7.021,12.886,0.000,7.021,1.554,1.254
3,1973-12-31,1973.0,25881.0,21.771,8.567,13.204,0.000,8.567,1.863,1.633
4,1974-12-31,1974.0,25881.0,25.638,9.843,15.381,0.414,10.257,1.555,1.313


Let's rename LPERMNO to PERMNO:

In [6]:
cstat.rename(columns={"LPERMNO":"PERMNO"}, inplace=True)
cstat.head(5)

,DATADATE,FYEAR,PERMNO,AT,CEQ,LT,PSTK,SEQ,IB,CAPX
0,1970-12-31,1970.0,25881.0,33.450,10.544,22.906,0.000,10.544,1.878,2.767
1,1971-12-31,1971.0,25881.0,29.330,8.381,20.948,0.000,8.382,0.138,1.771
2,1972-12-31,1972.0,25881.0,19.907,7.021,12.886,0.000,7.021,1.554,1.254
3,1973-12-31,1973.0,25881.0,21.771,8.567,13.204,0.000,8.567,1.863,1.633
4,1974-12-31,1974.0,25881.0,25.638,9.843,15.381,0.414,10.257,1.555,1.313


To use this data, we must make an assumption about the first date on which this data would be available.  Standard practice is to assume that by 6 months after the fiscal year end we will for sure have access to the annual report.  

I therefore create a new column, DATE, that is designed to represent the date that the data become known.  In the first line I set date equal to the fiscal year end plus six months.  In the second line I make sure that the date is the last day of the month.  As before, this will help when I merge this with the other datasets.

In [7]:
cstat['DATE'] = cstat['DATADATE'] + MonthEnd(0)
cstat.head()

,DATADATE,FYEAR,PERMNO,AT,CEQ,LT,PSTK,SEQ,IB,CAPX,DATE
0,1970-12-31,1970.0,25881.0,33.450,10.544,22.906,0.000,10.544,1.878,2.767,1970-12-31
1,1971-12-31,1971.0,25881.0,29.330,8.381,20.948,0.000,8.382,0.138,1.771,1971-12-31
2,1972-12-31,1972.0,25881.0,19.907,7.021,12.886,0.000,7.021,1.554,1.254,1972-12-31
3,1973-12-31,1973.0,25881.0,21.771,8.567,13.204,0.000,8.567,1.863,1.633,1973-12-31
4,1974-12-31,1974.0,25881.0,25.638,9.843,15.381,0.414,10.257,1.555,1.313,1974-12-31


With the date defined how I want it, we are ready to set indexes and sort:

In [8]:
cstat.set_index(['PERMNO','DATE'], inplace=True)
cstat.sort_index(inplace=True)
cstat.head()

DATADATE   FYEAR      AT    CEQ      LT  PSTK    SEQ  \
PERMNO  DATE                                                                
10000.0 1986-10-31 1986-10-31  1986.0   2.115  0.418   1.697   0.0  0.418   
10001.0 1986-06-30 1986-06-30  1986.0  12.242  5.432   6.810   0.0  5.432   
        1987-06-30 1987-06-30  1987.0  11.771  5.369   6.402   0.0  5.369   
        1988-06-30 1988-06-30  1988.0  11.735  5.512   6.223   0.0  5.512   
        1989-06-30 1989-06-30  1989.0  18.565  6.321  12.244   0.0  6.321   

                       IB   CAPX  
PERMNO  DATE                      
10000.0 1986-10-31 -0.730  0.240  
10001.0 1986-06-30  0.669  0.551  
        1987-06-30  0.312  0.513  
        1988-06-30  0.542  0.240  
        1989-06-30  1.208  0.444

We now need to merge these data.  Unfortunately, the data occasionally have multiple rows with the same PERMNO and DATE.  So we are going to have to eliminate duplicate PERMNO/DATE pairs.

There are many ways to do this.  My thought here is that we should assume that if there is more than one PERMNO on the same date, then the bigger one is probably more important and therefore more likely to be correct.

I am therefore going to sort the dataframe in ascending order by PERMNO, then in ascending order by DATE, and then in descending order by size (either MV or SEQ).  This is how you do it:

In [9]:
stocks = stocks.sort_values(by = ['PERMNO','DATE','MV'], ascending = [True, True, False])
cstat  = cstat.sort_values(by = ['PERMNO','DATE','SEQ'], ascending = [True, True, False])

Since the first observation for each PERMNO/DATE is the one I want to keep, I eliminate duplicates as follows:

In [10]:
stocks = stocks.groupby(['PERMNO','DATE']).head(1)
cstat  = cstat.groupby(['PERMNO','DATE']).head(1)

In [11]:
stocks.describe()

,RET,MV
count,4.110629e+06,4.145348e+06
mean,1.002869e-02,1.948814e+06
std,1.740034e-01,1.557125e+07
min,-9.936000e-01,0.000000e+00
25%,-5.975012e-02,2.349900e+04
50%,0.000000e+00,1.081290e+05
75%,6.269594e-02,5.490932e+05
max,2.400000e+01,2.902368e+09


Let's create momentum here.  I will first lag returns two months, and then I will compute the 11-month moving average of the lagged returns.

Note the use of droplevel in the second step.  

In [12]:
stocks['lag2 RET'] = stocks['RET'].groupby('PERMNO').shift(2)
stocks['momentum'] = stocks['lag2 RET'].groupby('PERMNO').rolling(11).mean().droplevel(0)

In the previous step, you will notice I used the droplevel method.  This is because of some unexpected behavior by groupby.  If I don't use droplevel, I get the following result:

In [13]:
stocks['lag2 RET'].groupby('PERMNO').rolling(11).mean()

PERMNO   PERMNO   DATE      
10000.0  10000.0  1985-12-31         NaN
                  1986-01-31         NaN
                  1986-02-28         NaN
                  1986-03-31         NaN
                  1986-04-30         NaN
                                  ...   
93436.0  93436.0  2022-02-28    0.035781
                  2022-03-31    0.038975
                  2022-04-30    0.033560
                  2022-05-31    0.049548
                  2022-06-30    0.042890
Name: lag2 RET, Length: 4285985, dtype: float64

For some reason, groupby creates a redundant PERMNO index.  This is eliminated by droplevel(0), which drops the first (which is 0) level of the index.

Now it's time to combine the CRSP and Compustat data.  Because of the way we have indexed each dataframe, specifically making sure that all dates are the last days of the month, this is super easy.  Just type:

stocks[['IB','SEQ']]   = cstat[['IB','SEQ']]

However, this is a bit slow.  A much faster alternative is to use the merge method:

In [14]:
stocks = stocks.merge(cstat[['IB','SEQ']], how='left', on=['PERMNO','DATE'])

To see what happened, let's take a look at one stock, Apple, which has PERMNO=14593:

In [15]:
stocks.describe()

,RET,MV,lag2 RET,momentum,IB,SEQ
count,4.110629e+06,4.145348e+06,4.066758e+06,3.693042e+06,280116.000000,279689.000000
mean,1.002869e-02,1.948814e+06,1.034960e-02,1.145380e-02,130.181297,1244.396579
std,1.740034e-01,1.557125e+07,1.729439e-01,4.901616e-02,1203.841510,7767.483739
min,-9.936000e-01,0.000000e+00,-9.936000e-01,-5.339999e-01,-99289.000000,-86154.000000
25%,-5.975012e-02,2.349900e+04,-5.947950e-02,-1.007911e-02,-0.897000,15.433000
50%,0.000000e+00,1.081290e+05,0.000000e+00,9.575736e-03,3.493000,75.325000
75%,6.269594e-02,5.490932e+05,6.308411e-02,3.070194e-02,32.205250,389.899000
max,2.400000e+01,2.902368e+09,2.400000e+01,2.158631e+00,94680.000000,506199.000000


In [16]:
stocks.loc[14593].tail(24)

,RET,MV,lag2 RET,momentum,IB,SEQ
DATE,,,,,,
2020-07-31,0.165132,1.817316e+09,0.084956,0.048177,NaN,NaN
2020-08-31,0.216309,2.206911e+09,0.147386,0.054631,NaN,NaN
2020-09-30,-0.102526,1.966079e+09,0.165132,0.071149,57411.0,65339.0
2020-10-31,-0.060012,1.850816e+09,0.216309,0.084181,NaN,NaN
2020-11-30,0.095490,2.024065e+09,-0.102526,0.064798,NaN,NaN
2020-12-31,0.114574,2.232279e+09,-0.060012,0.052304,NaN,NaN
2021-01-31,-0.005502,2.215357e+09,0.095490,0.052004,NaN,NaN
2021-02-28,-0.079532,2.035725e+09,0.114574,0.057510,NaN,NaN
2021-03-31,0.007340,2.038232e+09,-0.005502,0.067402,NaN,NaN


Apple's fiscal year ends in September.  That's why we see IB in those months and no others.

Now we will compute the E/P ratio in two different ways.

The first is to use the most recent known value of E (column IB) and divide it by the contemporaneous observation of P (column MV), meaning the value of MV that corresponds to the most recent fiscal year end.  We will then lag the ratio 6 months to account for the fact that earnings are not known for some time after the end of the quarter.

We will need to multiply the E/P ratio by 1000 for it to make sense.  The reason is that CRSP and Compustat are in different units.  In CRSP, the shares outstanding series used to create market values (MV) was in 1000s of shares.  Thus, the MV column is too small by a factor of 1000.  In Compustat, earnings (IB) are in millions of dollars.  Multiplying by 1000 makes these numbers comparable.


In [17]:
stocks['lag EP v1'] = stocks['IB'].groupby('PERMNO').shift(6) / stocks['MV'].groupby('PERMNO').shift(6) * 1000

Looking again at Apple, we can see what we have done:

In [18]:
stocks.loc[14593].tail(24)

,RET,MV,lag2 RET,momentum,IB,SEQ,lag EP v1
DATE,,,,,,,
2020-07-31,0.165132,1.817316e+09,0.084956,0.048177,NaN,NaN,NaN
2020-08-31,0.216309,2.206911e+09,0.147386,0.054631,NaN,NaN,NaN
2020-09-30,-0.102526,1.966079e+09,0.165132,0.071149,57411.0,65339.0,NaN
2020-10-31,-0.060012,1.850816e+09,0.216309,0.084181,NaN,NaN,NaN
2020-11-30,0.095490,2.024065e+09,-0.102526,0.064798,NaN,NaN,NaN
2020-12-31,0.114574,2.232279e+09,-0.060012,0.052304,NaN,NaN,NaN
2021-01-31,-0.005502,2.215357e+09,0.095490,0.052004,NaN,NaN,NaN
2021-02-28,-0.079532,2.035725e+09,0.114574,0.057510,NaN,NaN,NaN
2021-03-31,0.007340,2.038232e+09,-0.005502,0.067402,NaN,NaN,0.029201


The calculations look right, but they only result in one E/P ratio per year.  We will fill in the rest using the _fillna_ method with the _pad_ option.  This uses older data to fill in for missing values.  Because older data are used, we don't have to worry about look-ahead bias.  The groupby('PERMNO') step makes sure that we never fill in one firm's earnings with those of another firm.  The limit=15 option says that we will not use a prior value if it is more than 15 months old, which should be unusual situations.

In [19]:
stocks['lag EP v1']  = stocks['lag EP v1'].groupby('PERMNO').fillna(method='pad', limit=15)

The second approach will be to use the most recent known E divided by the most recent known P, even if these two variables are observed at very different times.

To start with this, lets compute the most recent E (column IB) that we would observe.  We'll then use fillna in the same way to fill in missing values with older data.

In [20]:
stocks['lag IB']  = stocks['IB'].groupby('PERMNO').shift(6).fillna(method='pad', limit=15)

Again taking a look at Apple, it seems to have worked as expected:

In [21]:
stocks.loc[14593].tail(24)

,RET,MV,lag2 RET,momentum,IB,SEQ,lag EP v1,lag IB
DATE,,,,,,,,
2020-07-31,0.165132,1.817316e+09,0.084956,0.048177,NaN,NaN,0.055525,55256.0
2020-08-31,0.216309,2.206911e+09,0.147386,0.054631,NaN,NaN,0.055525,55256.0
2020-09-30,-0.102526,1.966079e+09,0.165132,0.071149,57411.0,65339.0,0.055525,55256.0
2020-10-31,-0.060012,1.850816e+09,0.216309,0.084181,NaN,NaN,0.055525,55256.0
2020-11-30,0.095490,2.024065e+09,-0.102526,0.064798,NaN,NaN,0.055525,55256.0
2020-12-31,0.114574,2.232279e+09,-0.060012,0.052304,NaN,NaN,0.055525,55256.0
2021-01-31,-0.005502,2.215357e+09,0.095490,0.052004,NaN,NaN,0.055525,55256.0
2021-02-28,-0.079532,2.035725e+09,0.114574,0.057510,NaN,NaN,0.055525,55256.0
2021-03-31,0.007340,2.038232e+09,-0.005502,0.067402,NaN,NaN,0.029201,57411.0


To compute the E/P ratio in this approach, we divide the lagged IB column by the most recently observed value of MV, which is the value in the previous row:

In [22]:
stocks['lag EP v2'] = stocks['lag IB'] / stocks['MV'].groupby('PERMNO').shift(1) * 1000

Again, Apple:

In [23]:
stocks.loc[14593].tail(24)

,RET,MV,lag2 RET,momentum,IB,SEQ,lag EP v1,lag IB,lag EP v2
DATE,,,,,,,,,
2020-07-31,0.165132,1.817316e+09,0.084956,0.048177,NaN,NaN,0.055525,55256.0,0.035357
2020-08-31,0.216309,2.206911e+09,0.147386,0.054631,NaN,NaN,0.055525,55256.0,0.030405
2020-09-30,-0.102526,1.966079e+09,0.165132,0.071149,57411.0,65339.0,0.055525,55256.0,0.025038
2020-10-31,-0.060012,1.850816e+09,0.216309,0.084181,NaN,NaN,0.055525,55256.0,0.028105
2020-11-30,0.095490,2.024065e+09,-0.102526,0.064798,NaN,NaN,0.055525,55256.0,0.029855
2020-12-31,0.114574,2.232279e+09,-0.060012,0.052304,NaN,NaN,0.055525,55256.0,0.027300
2021-01-31,-0.005502,2.215357e+09,0.095490,0.052004,NaN,NaN,0.055525,55256.0,0.024753
2021-02-28,-0.079532,2.035725e+09,0.114574,0.057510,NaN,NaN,0.055525,55256.0,0.024942
2021-03-31,0.007340,2.038232e+09,-0.005502,0.067402,NaN,NaN,0.029201,57411.0,0.028202


Note that the two E/P ratios are not the same.

Our primary analysis will be on the RET, lag EP v1, and lag EP v2 columns.  Let's make sure all three variables are observed:

In [24]:
stocks = stocks.dropna(subset=['RET','lag EP v1','lag EP v2','momentum'])

In [25]:
stocks.describe()

,RET,MV,lag2 RET,momentum,IB,SEQ,lag EP v1,lag IB,lag EP v2
count,2.999368e+06,2.999368e+06,2.999368e+06,2.999368e+06,246477.000000,246095.000000,2.999368e+06,2.999368e+06,2.999368e+06
mean,1.208671e-02,2.446027e+06,1.210856e-02,1.252202e-02,145.539061,1375.434899,5.806177e-02,1.342925e+02,-5.799284e-02
std,1.828115e-01,1.803403e+07,1.799087e-01,5.160033e-02,1274.916193,8236.496645,1.963338e+01,1.151458e+03,2.214101e+01
min,-9.936000e-01,4.156250e+00,-9.936000e-01,-5.227778e-01,-99289.000000,-86154.000000,-5.922079e+03,-9.928900e+04,-8.497474e+03
25%,-6.548918e-02,2.772400e+04,-6.557377e-02,-1.193139e-02,-0.788000,18.500500,-1.517238e-02,-4.750000e-01,-1.370171e-02
50%,0.000000e+00,1.349674e+05,0.000000e+00,1.084311e-02,4.502000,89.187000,4.968467e-02,4.231000e+00,4.540851e-02
75%,7.142857e-02,7.515121e+05,7.168459e-02,3.373189e-02,39.800000,456.653000,9.511934e-02,3.582500e+01,9.183478e-02
max,2.400000e+01,2.902368e+09,2.400000e+01,2.158631e+00,94680.000000,506199.000000,4.663291e+03,9.468000e+04,2.026765e+03


Since we are going to combine different firms, on the same date, into portfolios, the next step is to sort by DATE first and then by PERMNO.

In [26]:
stocks = stocks.reorder_levels(['DATE','PERMNO'])
stocks.sort_index(inplace=True)

One final step before portfolios are constructed is to eliminate observations that look funny.  These could be the result of earnings being close to zero.  They could also be database errors or errors in merging the different databases.  

In any case, it is entirely feasible for an investor to say that he or she is not going to hold stocks that have P/E ratios below 5 or above 100, so I will exclude all stocks that are outside that range (using either P/E measure).  This gets rid of a lot of strange observations:

In [27]:
stocks = stocks.loc[(stocks['lag EP v1']>0) & (stocks['lag EP v2']>0) & (stocks['lag EP v1']<.5) & (stocks['lag EP v2']<.5) ]

Finally, time to compute portfolios.  I will compute quintile portfolios similarly to how we did it in the reversal strategy.  Everything from here to the end follows almost exactly from the more_reversal folder.  I now have two sets of portfolios, however, and two sets of statistics, one for each of my P/E measures:

In [28]:
def quintiles(inser):
    outser = pd.qcut(inser, q=5, labels=range(1,6))
    return outser

We're going to look at quintiles formed on two different measures.  Rather than type all the code twice, let's create functions that do all the necessary calculations.  The first one examines quintile portfolios and returns the HML long/short portfolio.  The second computes performance statistics for the HML portfolio returns.

In [29]:
# the function takes the name of some variable we want to use as its single input
def perf(sortvar):
    
    # creating the quintiles
    stocks['Q'] = stocks[sortvar].groupby('DATE').apply(quintiles)
    
    # computing quintile portfolio returns
    ports = stocks.groupby(['Q','DATE'])['RET'].mean()
    
    # printing out basic statistics on each portfolio
    print(ports.groupby('Q').describe())
    
    # computing high minus low portfolios
    hml = ports.loc[5] - ports.loc[1]
    
    return hml

def stats(hml):
    # printing basic statistics plus sharpe and t-stat
    stats = hml.describe()
    stats.loc['tstat']  = stats.loc['mean'] / stats.loc['std'] * np.sqrt(stats.loc['count'])
    stats.loc['sharpe'] = stats.loc['mean'] / stats.loc['std'] * np.sqrt(12)    
    print(stats)

Now we just have to call the function twice:

In [30]:
hml_ep = perf('lag EP v2')
stats(hml_ep)

   count      mean       std       min       25%       50%       75%       max
Q                                                                             
1  618.0  0.009976  0.058833 -0.310993 -0.023765  0.014277  0.044461  0.215154
2  618.0  0.010424  0.051543 -0.295033 -0.016762  0.013854  0.042004  0.240039
3  618.0  0.011241  0.049082 -0.268957 -0.015260  0.014905  0.038250  0.283728
4  618.0  0.013657  0.050063 -0.253019 -0.011663  0.016707  0.038712  0.297150
5  618.0  0.014535  0.060294 -0.309199 -0.014436  0.015657  0.042747  0.370504
count     618.000000
mean        0.004558
std         0.032621
min        -0.197925
25%        -0.012882
50%         0.003005
75%         0.019977
max         0.178392
tstat       3.473768
sharpe      0.484058
Name: RET, dtype: float64


Either way, higher E/P stocks have higher average returns, but this is slightly more the case when we use the second version of the E/P ratio, which is based on the more current market values.  However, this slightly higher mean comes with a higher SD as well.  As a result, the second E/P ratio has a slightly lower Sharpe ratio.  

In [31]:
hml_mom = perf('momentum')
stats(hml_mom)

   count      mean       std       min       25%       50%       75%       max
Q                                                                             
1  618.0  0.007654  0.067506 -0.285558 -0.028373  0.007157  0.040700  0.376199
2  618.0  0.010509  0.050812 -0.241598 -0.013881  0.011720  0.035108  0.296705
3  618.0  0.012161  0.047126 -0.263612 -0.011820  0.015720  0.037822  0.260857
4  618.0  0.013413  0.048237 -0.281059 -0.011739  0.017223  0.041082  0.240278
5  618.0  0.016102  0.059269 -0.309417 -0.017930  0.019281  0.054548  0.233996
count     618.000000
mean        0.008447
std         0.042172
min        -0.303440
25%        -0.008989
50%         0.012463
75%         0.030561
max         0.164047
tstat       4.979509
sharpe      0.693878
Name: RET, dtype: float64


One way to combine the value and momentum strategies is just to put half of your money in each long/short portfolio.  The performance of this strategy is as follows:

In [32]:
stats(.5*hml_ep + .5*hml_mom)

count     618.000000
mean        0.006503
std         0.014854
min        -0.067197
25%        -0.000539
50%         0.006209
75%         0.014497
max         0.061864
tstat      10.882924
sharpe      1.516499
Name: RET, dtype: float64


An alternative way to combine signals is to normalize them and take the sum, i.e.
$$ score_{i,t} = \frac{ep_{i,t} - \mu^{ep}_t}{\sigma^{ep}_t} + \frac{mom_{i,t} - \mu^{mom}_t}{\sigma^{mom}_t} ,$$
where $\mu^x_t$ and $\sigma^x_t$ are the mean and SD of $x$ across all firms at date $t$. 

Note that subtracting the means affects the score, but it does not change the rankings of different stocks.  So in the next calculation, I don't bother to remove them.

In [33]:
stocks['score'] = stocks['lag EP v2']/stocks['lag EP v2'].groupby('DATE').std() + stocks['momentum']/stocks['momentum'].groupby('DATE').std()

Now I form quintile portfolios based on $score$:

In [34]:
hml_score = perf('score')
stats(hml_score)

   count      mean       std       min       25%       50%       75%       max
Q                                                                             
1  618.0  0.006130  0.061235 -0.290881 -0.027001  0.007532  0.038472  0.280619
2  618.0  0.009930  0.049859 -0.267899 -0.017396  0.011884  0.037538  0.268785
3  618.0  0.012351  0.047718 -0.254217 -0.012640  0.015912  0.039860  0.263510
4  618.0  0.014764  0.049655 -0.252878 -0.011764  0.018129  0.043290  0.279544
5  618.0  0.016661  0.058949 -0.284216 -0.013234  0.019582  0.049903  0.291496
count     618.000000
mean        0.010531
std         0.024629
min        -0.101107
25%        -0.000506
50%         0.010567
75%         0.024076
max         0.102245
tstat      10.629563
sharpe      1.481194
Name: RET, dtype: float64


Another possibility is to do a two-way sort.  I will do independent quintile sorts on EP and momentum.  I will then go long stocks that are in the 5th quintile of both variables and short stocks that are in both 1st quintiles.

In [35]:
# creating the quintiles
stocks['Qep']  = stocks['lag EP v2'].groupby('DATE').apply(quintiles)
stocks['Qmom'] = stocks['momentum'].groupby('DATE').apply(quintiles)

# computing quintile portfolio returns
ports = stocks.groupby(['Qep','Qmom','DATE'])['RET'].mean()

# printing out basic statistics on each portfolio
print(ports.groupby(['Qep','Qmom']).describe())

# computing high minus low portfolios
hml = ports.loc[5,5] - ports.loc[1,1]

stats(hml)

          count      mean       std       min       25%       50%       75%  \
Qep Qmom                                                                      
1   1     618.0  0.002303  0.070856 -0.321186 -0.035458  0.002560  0.037331   
    2     618.0  0.006889  0.057491 -0.275156 -0.026785  0.009813  0.039833   
    3     618.0  0.008199  0.053490 -0.300099 -0.023690  0.011540  0.038190   
    4     618.0  0.010267  0.053985 -0.310271 -0.020138  0.012856  0.042033   
    5     618.0  0.015401  0.065683 -0.322213 -0.021716  0.019799  0.057309   
2   1     618.0  0.004661  0.068243 -0.313105 -0.031290  0.005198  0.041251   
    2     618.0  0.008115  0.052818 -0.275984 -0.018658  0.009835  0.037503   
    3     618.0  0.009910  0.048380 -0.270942 -0.016357  0.014147  0.036912   
    4     618.0  0.011821  0.048796 -0.287718 -0.013692  0.014649  0.042749   
    5     618.0  0.014647  0.056972 -0.314213 -0.017360  0.017836  0.052476   
3   1     618.0  0.006624  0.064651 -0.268897 -0.028

Finally, I will try FM regression.

In [36]:
# just renaming a variable so it doesn't have spaces
stocks['lag_EP_v2'] = stocks['lag EP v2']

In [37]:
def regfun(df, mod):
    results = smf.ols(mod, data=df).fit()
    return results.params

In [38]:
allparams = stocks.groupby('DATE').apply(regfun, 'RET ~ lag_EP_v2 + momentum')

Predictions need to be made based on lagged data.  Thus, I can only use lagged data when running the regression.  This means that the rows in allparams that I am allowed to use are those that precede the row I am forecasting.

I can use some or all of these rows.  That is up to me.  Here, I choose to use the most recent 10 years of data (120 months) to construct my FM coefficients.

In [39]:
rollparams = allparams.rolling(120).mean().shift()

Renaming the coefficients so I can merge them into the main dataframe:

In [40]:
rollparams = rollparams.rename(columns={'lag_EP_v2':'lag_EP_v2 coef', 'momentum':'momentum coef'})
rollparams.tail()

,Intercept,lag_EP_v2 coef,momentum coef
DATE,,,
2022-02-28,0.010135,0.004581,0.045622
2022-03-31,0.009873,0.004724,0.046685
2022-04-30,0.009728,0.004998,0.046287
2022-05-31,0.009165,0.005793,0.046218
2022-06-30,0.009709,0.007394,0.043142


Merging coefficients with data:

In [41]:
stocks = stocks.merge(rollparams, how='left', on='DATE')

Computing forecasts:

In [42]:
stocks['fmpred'] = stocks['Intercept'] + stocks['lag_EP_v2']*stocks['lag_EP_v2 coef'] + stocks['momentum']*stocks['momentum coef']

Doing a quintile sort of returns based on the FM forecasts:

In [49]:
stocks.head()

,RET,MV,lag2 RET,momentum,IB,SEQ,lag EP v1,lag IB,lag EP v2,Q,score,Qep,Qmom,lag_EP_v2,Intercept,lag_EP_v2 coef,momentum coef,fmpred
DATE,,,,,,,,,,,,,,,,,,
1981-01-31,0.108696,459459.000,0.120235,0.040548,NaN,NaN,0.157672,47.841,0.115443,3,2.497342,3,4,0.115443,0.006507,0.044536,0.182109,0.019032
1981-01-31,-0.009804,107893.250,0.030000,0.013504,NaN,NaN,0.194032,19.637,0.180220,3,2.689568,4,2,0.180220,0.006507,0.044536,0.182109,0.016992
1981-01-31,0.009091,558996.000,0.036036,0.003136,NaN,NaN,0.159982,98.765,0.178289,2,2.412546,4,1,0.178289,0.006507,0.044536,0.182109,0.015018
1981-01-31,-0.065421,1674450.000,0.059729,0.022677,NaN,NaN,0.124112,175.643,0.098034,2,1.835290,3,3,0.098034,0.006507,0.044536,0.182109,0.015002
1981-01-31,-0.185915,7787.625,0.043478,-0.003963,NaN,NaN,0.131314,2.282,0.199169,2,2.513800,5,1,0.199169,0.006507,0.044536,0.182109,0.014655


In [43]:
stocks = stocks.dropna(subset=['RET','fmpred'])
hml_fm = perf('fmpred')

   count      mean       std       min       25%       50%       75%       max
Q                                                                             
1  498.0  0.006257  0.062163 -0.296964 -0.026706  0.008491  0.039073  0.323563
2  498.0  0.009755  0.046689 -0.251669 -0.013476  0.013138  0.035703  0.189695
3  498.0  0.012168  0.043843 -0.249033 -0.010377  0.016848  0.036584  0.168249
4  498.0  0.013032  0.044121 -0.270471 -0.009743  0.017445  0.039764  0.131016
5  498.0  0.015016  0.054418 -0.302499 -0.012184  0.019816  0.048428  0.178923


In [44]:
stats(hml_fm)

count     498.000000
mean        0.008759
std         0.034477
min        -0.190323
25%        -0.006133
50%         0.010855
75%         0.027381
max         0.141000
tstat       5.669410
sharpe      0.880063
Name: RET, dtype: float64


The results are not so great.  One possible reason is that my implementation of FM required at least 10 years of data before making the first prediction.  Perhaps the bad performance is due to a different sample period.  To check, I will look at the simple 50/50 strategy over the same sample period.

In [45]:
stats(.5*hml_ep.iloc[120:] + .5*hml_mom.iloc[120:]) 
# 앞에 rolling이랑 dropna 때문에 빠진 10년을 제외한 기간에 대해서 50/50 strategy 적용해보자 (FM과 같은 sample period)
# Sharpe ratio 1.44로 앞에 FM model의 0.88 보다 높게 나온다. FM 모델이 별로였다는 결론

count     498.000000
mean        0.006213
std         0.014922
min        -0.067197
25%        -0.001083
50%         0.005949
75%         0.014278
max         0.061864
tstat       9.292099
sharpe      1.442414
Name: RET, dtype: float64


This looks OK, so the problem was FM, not the sample period.